In [10]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [11]:
df = pd.read_csv("data/all_data.csv")

C:\Users\lenovo\AppData\Local\Temp\ipykernel_92588\3684764311.py:1: DtypeWarning: Columns (3,55,88,94) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/all_data.csv")


In [12]:
df

,Unnamed: 0,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 42,Unnamed: 43,Unnamed: 39,Unnamed: 40
0,0,D1,2003/8/1,NaN,Bayern Munich,Ein Frankfurt,3,1,H,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,F1,2003/8/1,NaN,Lille,Lyon,1,0,H,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,F1,2003/8/2,NaN,Auxerre,Nice,1,2,A,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,F1,2003/8/2,NaN,Guingamp,Marseille,0,1,A,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,D1,2003/8/2,NaN,Hamburg,Hannover,0,3,A,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39181,39181,E0,2025/5/25,16:00,Tottenham,Brighton,1,4,A,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39182,39182,I1,2025/5/25,19:45,Udinese,Fiorentina,2,3,A,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39183,39183,I1,2025/5/25,19:45,Venezia,Juventus,2,3,A,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39184,39184,SP1,2025/5/25,15:15,Villarreal,Sevilla,4,2,H,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# remove unknown columns
columns = df.columns
unknown_cols = [c for c in columns if "Unnamed" in c]
df = df.drop(columns=unknown_cols)

# remove rows with only na values
df = df.dropna(how="all")

features_to_keep = ["Div", "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HS", "AS", "HST", "AST", 
                    "B365H", "B365D", "B365A",
                    "WHH", "WHD", "WHA",
                    "AHh", "B365AHH", "B365AHA",
                    "AHCh", "B365CAHH", "B365CAHA",
                    "B365>2.5", "B365<2.5",
                    ]
df = df[features_to_keep]

In [14]:
name_conversions = {
'St Etienne': 'Saint-Etienne',
'Ajaccio GFCO': 'Ajaccio', 
'Ath Madrid': 'Atletico',
'Ein Frankfurt': 'Frankfurt',
'Espanol': 'Espanyol',
'La Coruna': 'Depor',
'Nurnberg': 'Nuernberg',
'M\'gladbach': 'Gladbach',
'Bayern Munich': 'Bayern',
'Greuther Furth': 'Fuerth',
'Sp Gijon': 'Gijon',
'FC Koln': 'Koeln',
'Ath Bilbao': 'Bilbao',
'Hansa Rostock': 'Rostock',
'Fortuna Dusseldorf': 'Duesseldorf',
'Schalke 04': 'Schalke',
'Werder Bremen': 'Werder',
'Vallecano': 'RayoVallecano',
'Kaiserslautern': 'Lautern',
"Nott'm Forest": 'Forest',
'Holstein Kiel': 'Holstein',
'Munich 1860': 'Munich'
}

In [15]:
from os.path import exists
all_teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))


In [16]:

club_elo_base_url = "http://api.clubelo.com/"
name_mismatches = []
exceptions = []
for team_name in all_teams:
    try:
        # if exists("elo/" + team_name + ".csv"):
        #     continue
        team_name_url = name_conversions.get(team_name, team_name).replace(" ", "")
        team_data = requests.get(club_elo_base_url + team_name_url, stream=True)
        byte_data = team_data.content
        if team_name == "Nott'm Forest":
            a = 1
        if len(byte_data) == 38 or team_data.status_code != 200:
            # means that only header was returned, so team was not found
            name_mismatches.append(team_name)
            continue
        with open("elo/" + team_name + ".csv", mode='wb') as f:
            f.write(team_data.content)
    except:
        exceptions.append(team_name)
        continue
print("Name mismatches: ", name_mismatches)
print("Exceptions: ", exceptions)

Name mismatches:  ['Arles', 'Verona', 'Grenoble', 'Reading', 'QPR', 'Munich 1860', 'Evian Thonon Gaillard', 'Gimnastic']
Exceptions:  []


In [17]:
import datetime
from functools import partial

# for team in all_teams:
#     df.loc[(df["HomeTeam"] == team) | (df["AwayTeam"] == team)]
# # df = df.apply(add_elo_feature, axis=1)
# df.to_csv("data/all_data_with_elo.csv")
# df

dfs = {}

for team in all_teams:
    if exists("elo/" + team + ".csv"):
        dfs[team] = pd.read_csv("elo/" + team + ".csv")
        
def find_team_elo_at(team_name, date):
    elo_df = dfs.get(team_name, None)
    if elo_df is None:
        return None
    elo_df["From"] = pd.to_datetime(elo_df["From"], format='mixed', dayfirst=True)
    elo_df["To"] = pd.to_datetime(elo_df["To"], format='mixed', dayfirst=True)
    if date > '2000-05-01':
        aa = 1
    # if date > '2024-10-01':
    #     aa = 2
    try:
        # elo = elo_df.loc[elo_df["From"] >= date]["Elo"].iloc[0]
        # print(elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] > date)])
        elo = elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] >= date)]["Elo"].iloc[0]
        return elo
    except:
        return None
    
def add_elo_feature(s):
    s["HomeTeamELO"] = find_team_elo_at(s["HomeTeam"], s["Date"])
    s["AwayTeamELO"] = find_team_elo_at(s["AwayTeam"], s["Date"])
    return s

f = np.vectorize(find_team_elo_at)

df["HomeTeamELO"] = f(df["HomeTeam"], df["Date"])
df["AwayTeamELO"] = f(df["AwayTeam"], df["Date"])

# df = df.apply(add_elo_feature, axis=1)
df.to_csv("data/all_data_with_elo.csv")
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,...,AHh,B365AHH,B365AHA,AHCh,B365CAHH,B365CAHA,B365>2.5,B365<2.5,HomeTeamELO,AwayTeamELO
0,D1,2003/8/1,Bayern Munich,Ein Frankfurt,3,1,H,17.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1859.379272,1593.249268
1,F1,2003/8/1,Lille,Lyon,1,0,H,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1612.968018,1726.539795
2,F1,2003/8/2,Auxerre,Nice,1,2,A,NaN,NaN,NaN,...,-0.75,2.050,1.850,NaN,NaN,NaN,NaN,NaN,1702.604858,1611.196045
3,F1,2003/8/2,Guingamp,Marseille,0,1,A,NaN,NaN,NaN,...,0.00,1.925,1.975,NaN,NaN,NaN,NaN,NaN,1685.016113,1665.625732
4,D1,2003/8/2,Hamburg,Hannover,0,3,A,10.0,16.0,NaN,...,-0.75,1.800,2.100,NaN,NaN,NaN,NaN,NaN,1718.566284,1649.805298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39181,E0,2025/5/25,Tottenham,Brighton,1,4,A,4.0,23.0,2.0,...,0.75,2.000,1.850,0.50,1.95,1.90,1.33,3.40,1786.625366,1809.799683
39182,I1,2025/5/25,Udinese,Fiorentina,2,3,A,12.0,25.0,3.0,...,0.50,1.850,2.000,0.25,1.80,2.05,1.98,1.88,1631.991089,1756.727539
39183,I1,2025/5/25,Venezia,Juventus,2,3,A,12.0,15.0,5.0,...,1.00,1.900,1.950,1.25,1.85,2.00,1.88,1.98,1567.002563,1807.898438
39184,SP1,2025/5/25,Villarreal,Sevilla,4,2,H,12.0,17.0,7.0,...,-1.25,2.000,1.850,-0.75,1.95,1.90,1.53,2.50,1783.361816,1644.167236


In [18]:
df["Date"][10000] > '2024-10-01'

False